In [2]:
import urllib.request as req
from bs4 import BeautifulSoup

In [7]:
url_index = "https://myindex.jp/"

In [8]:
def get_link_urls(url):
    res = req.urlopen(url)
    soup = BeautifulSoup(res, "html.parser")
    div = soup.find('div', id='tab_searchf_1')
    table = div.find('table', class_='tabstyle')
    rows = table.find_all('tr')
    
    # tableのlink先取得
    link_urls = []
    for row in rows:
        columns = row.find_all("td")
        if len(columns) >= 3:
            link = columns[2].find("a")
            if link:
                href = link["href"]
                link_url = url_index + href
                link_urls.append(link_url)

    return link_urls    

In [9]:
link_urls = get_link_urls("https://myindex.jp/search_fund.php")
link_urls

['https://myindex.jp/data_f.php?q=1319',
 'https://myindex.jp/data_f.php?q=1364',
 'https://myindex.jp/data_f.php?q=1474',
 'https://myindex.jp/data_f.php?q=1551',
 'https://myindex.jp/data_f.php?q=1591',
 'https://myindex.jp/data_f.php?q=1592',
 'https://myindex.jp/data_f.php?q=1593',
 'https://myindex.jp/data_f.php?q=1599',
 'https://myindex.jp/data_f.php?q=2042',
 'https://myindex.jp/data_f.php?q=2516',
 'https://myindex.jp/data_f.php?q=2526',
 'https://myindex.jp/data_f.php?q=DMJP',
 'https://myindex.jp/data_f.php?q=EWJ',
 'https://myindex.jp/data_f.php?q=01311143',
 'https://myindex.jp/data_f.php?q=0131A141',
 'https://myindex.jp/data_f.php?q=02317141',
 'https://myindex.jp/data_f.php?q=03311144',
 'https://myindex.jp/data_f.php?q=04313141',
 'https://myindex.jp/data_f.php?q=04314141',
 'https://myindex.jp/data_f.php?q=04316141']

In [10]:
def get_attibute(soup):
    attribute = []
    div = soup.find('div', id='indexinfo_gaiyo')
    table = div.find('table')
    rows = table.find_all('tr')
    
    for row in rows:
        row_text = [td.get_text(strip=True) for td in row.select('th, td')]
        if any(keyword in row_text[0] for keyword in ["銘柄名", "大カテゴリ", "連動インデックス", "信託報酬"]):
            attribute.append(row_text[1])
    return attribute


def get_performance(soup):
    performance = []
    div = soup.find('div', class_='indexinfo_pfm w99')
    table = div.find('table')
    rows = table.find_all('tr')
    
    # リスク、リターン、シャープレシオを抽出
    for row in rows:
        if row.find('td'):
            row_text = [td.get_text(strip=True) for td in row.select('td')]
            if any(keyword in row_text[0] for keyword in ["リスク", "リターン", "シャープレシオ"]):
                # print(row_text)
                # 最長期間のデータのみを抽出
                for i, elm in enumerate(reversed(row_text)):
                    if elm:
                        performance.append(elm)
                        break
                    if i >6:      # 過去データが1年以上ある銘柄のみを抽出
                        break
    return performance


for link_url in link_urls:
    response = req.urlopen(link_url)
    soup = BeautifulSoup(response, "html.parser")
    
    attribute = get_attibute(soup)
    performance = get_performance(soup)
    print(attribute, performance)
    
    

    break

['NEXT FUNDS 日経300株価指数連動型上場投資信託 (1319)', '日経株価指数300', '日本株式', '0.52%\u3000(年率・税抜)'] ['+6.6', '16.8', '0.4']


In [13]:
performance

['+6.6', '16.8', '0.4']